# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

### Start by importing libraries

In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy # if not yet installed
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium # if not yet installed
import folium # map rendering library

#!pip install bs4 # Uncomment if bs4 is not installed
from bs4 import BeautifulSoup # import BeautifulSoup

print('Libraries imported.')

### Creates url and saves it as a BeautifulSoup object

In [ ]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(website_url.content,'html.parser')

In [ ]:
#print(soup.prettify()) # Uncomment if you want to see the html data

### Saving raw html data into a panda dataframe

In [ ]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

### Print first 5 entries of data frame and shape

In [ ]:
df.head()

In [ ]:
df.shape

### Creating a new dataframe and reading in csv file for coordinates since geocoder does not work at the moment

In [ ]:
#Creating new df
column_names = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=column_names)

#Importing coordinate df
coordinates = pd.read_csv('Geospatial_Coordinates.csv')

### Iterating through each entry and checking for correct coordinates and adding to new dataframe

In [ ]:
# Iterating through neighborhood df
for i in range(len(df)):

    postalCode = df.iloc[i]['PostalCode']
    borough = df.iloc[i]['Borough'] 
    neighborhood_name = df.iloc[i]['Neighborhood']
    n_lon = None
    n_lat = None

    # Iterating through coordinates df
    for j in range(len(coordinates)):
        if postalCode == coordinates.loc[j][0]:
            n_lat = coordinates.loc[j][1]
            n_lon = coordinates.loc[j][2]

    # Appending data to new df
    neighborhoods = neighborhoods.append({'PostalCode': postalCode,
                                          'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': n_lat,
                                          'Longitude': n_lon}, ignore_index=True)

### New dataframe first 5 entries and shape

In [ ]:
neighborhoods.head()

In [ ]:
neighborhoods.shape

### Getting Coordinates for Toronto

In [ ]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada are {}, {}.'.format(latitude, longitude))

### Using Folium to create a map with neighborhoods marked

In [ ]:
# Creating a map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# Adding markers to map
for lat, lng, label in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## I will compare the neighborhoods in regards to venues using K-means

### Defining parameters for Foursquare extraction

In [ ]:
CLIENT_ID = 'VQDQPVLJJG2IMTHHORGUYVPFNIL2J2LHCCJE2JX2UFJH0E2Y' # your Foursquare ID
CLIENT_SECRET = 'WP3N1IYMKRQRPOFEK44BCIIOMLZPB1WK11JOXNEFBXK1HDSP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
radius = 500

### Function for getting information about venues

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT) 
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']  
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

### Using function and saving results in variable

In [ ]:
toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )


In [ ]:
toronto_venues.head()

In [ ]:
toronto_venues.shape

In [ ]:
# One hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# Adding a neighborhood column to the dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# Moving neighborhood column to the first column
cols = list(toronto_onehot)
cols.insert(0, cols.pop(cols.index('Neighborhood')))
toronto_onehot = toronto_onehot.loc[:, cols]
toronto_onehot.head()


In [ ]:
# Grouping venues based on neighborhood
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [ ]:
# Printing top 5 venues per neighborhood
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
# Function for sorting venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# Creating columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# Creating a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

### Using K-means clustering for analysis

In [ ]:
# Clustering using K-means

#Using 7 clusters
kclusters = 7

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# Running k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# Labels generated
kmeans.labels_

In [ ]:
# Adding clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighborhoods

# Merging toronto_merged with neighborhood data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

# Dropping NaN rows
toronto_merged.dropna(subset = ['Cluster Labels'], inplace=True)

In [ ]:
# Converting label column to integer since it for some reason decided to be float
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)

In [ ]:
# Another folium map for visualizing clusters

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Visual results
As can be seen in the visualized results: there are no geographical indications of where similar neighborhoods are located, nor are they geographically grouped.
Most neighborhoods fall under the same category with a few exceptions. We can however note thtat most neighborhood that are different from the majority (marked in red) are located outside the center of the city.
